In [12]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client.from_service_account_json('/Users/servandodavidtorresgarcia/servando/relu/gcp/local-catalyst-398309-52645c242657.json')


# Define your table and destination URI
dataset_name = 'thelook_ecommerce'
table_name = 'distribution_centers'
bucket_name = 'ecommerce_looker_download'
destination_uri = f"gs://{bucket_name}/{table_name}.csv"

# Create an extract job
dataset_ref = client.dataset(dataset_name, project='bigquery-public-data')
table_ref = dataset_ref.table(table_name)

extract_job = client.extract_table(
    table_ref,
    destination_uri,
    location='US',  # Location must match that of the source table.
)
extract_job.result()  # Wait for job to complete

print(f"Exported {dataset_name}.{table_name} to {destination_uri}")


Exported thelook_ecommerce.distribution_centers to gs://ecommerce_looker_download/distribution_centers.csv


In [1]:

from src.data.dataset_reader import DatasetReader
from pathlib import Path
import pandas as pd
pd.set_option('display.max_columns', None)
%load_ext autoreload
%autoreload 2
# ignore warnings
import warnings
warnings.filterwarnings('ignore')
data_folder = Path("../data/raw").resolve()

products = DatasetReader(data_folder).get_data_csv('products.csv')
orders = DatasetReader(data_folder).get_data_csv('orders.csv')
order_items = DatasetReader(data_folder).get_data_csv('order_items.csv')
inventory_items = DatasetReader(data_folder).get_data_csv('inventory_items.csv')

In [34]:
order_items.query('id == 27569')

,id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
162087,27569,18975,15214,19220,74342,Processing,2023-01-05 04:51:42 UTC,NaN,NaN,NaN,53.549999


In [28]:
orders.query("order_id == 18975")

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
114110,18975,15214,Processing,M,2023-01-05 04:59:00 UTC,NaN,NaN,NaN,1


In [31]:
products.query("id == 19220")

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
6443,19220,26.9892,Sweaters,Nautica Men's Intarsia Argyle Vest Sweater,Nautica,53.549999,Men,29DC30AD86F873300D82DF7EDE9A0441,8


In [33]:
inventory_items.query("id == 74342")

,id,product_id,created_at,sold_at,cost,product_category,product_name,product_brand,product_retail_price,product_department,product_sku,product_distribution_center_id
148794,74342,19220,2021-05-29 12:31:00 UTC,NaN,26.9892,Sweaters,Nautica Men's Intarsia Argyle Vest Sweater,Nautica,53.549999,Men,29DC30AD86F873300D82DF7EDE9A0441,8


In [2]:
# Select specific columns from each DataFrame
order_items_selected = order_items[['product_id','order_id','status', 'created_at', 'shipped_at', 'delivered_at', 'returned_at', 'sale_price']]
orders_selected = orders[['order_id', 'gender', 'num_of_item']]  # Including 'order_id' for joining
products_selected = products[['id', 'cost', 'category', 'name', 'brand', 'retail_price', 'department', 'sku']]  # Including 'id' for joining

# Perform the join operations based on the relationships identified in the queries
# Assuming 'order_id' is common between 'order_items' and 'orders'
# Assuming 'product_id' is common between 'order_items' and 'products'

merged_df = order_items_selected \
    .merge(orders_selected, on='order_id', how='inner') \
    .merge(products_selected, left_on='product_id', right_on='id', how='inner')



In [3]:
merged_df.drop(columns=['product_id', 'order_id', 'id'], inplace=True)

In [4]:
# place sku to the beginning of the dataframe
cols = list(merged_df.columns)
cols.insert(0, cols.pop(cols.index('sku')))
merged_df = merged_df.loc[:, cols]


In [5]:
merged_df.head()
# checl if sale_price is equal to retail_price
merged_df.query('sale_price != retail_price')
merged_df.drop(columns=['retail_price'], inplace=True)

In [6]:
print(merged_df.head().to_string(index=False))

                             sku   status                     created_at                     shipped_at                   delivered_at returned_at  sale_price gender  num_of_item    cost    category                                    name            brand department
4ACBEDBE977480D19B7B682D4878CAE2  Shipped        2023-08-20 21:21:15 UTC        2023-08-21 07:58:00 UTC                            NaN         NaN         2.5      F            4  1.1075 Accessories Elegant PASHMINA SCARF WRAP SHAWL STOLE Scarf_tradinginc      Women
4ACBEDBE977480D19B7B682D4878CAE2 Complete 2023-09-08 00:38:23.869168 UTC 2023-09-07 19:56:53.869168 UTC 2023-09-08 14:07:53.869168 UTC         NaN         2.5      F            4  1.1075 Accessories Elegant PASHMINA SCARF WRAP SHAWL STOLE Scarf_tradinginc      Women
E0F19F64F086E393CEB0CF4A8C561B51  Shipped        2023-08-24 21:44:30 UTC        2023-08-21 07:58:00 UTC                            NaN         NaN        64.0      F            4 30.2080      Shorts 

In [58]:
merged_df.query('sku == "4ACBEDBE977480D19B7B682D4878CAE2"')

,sku,status,created_at,shipped_at,delivered_at,returned_at,sale_price,gender,num_of_item,cost,category,name,brand,department,sales
0,4ACBEDBE977480D19B7B682D4878CAE2,Shipped,2023-08-20 21:21:15+00:00,2023-08-21 07:58:00+00:00,NaT,NaT,2.5,F,4,1.1075,Accessories,Elegant PASHMINA SCARF WRAP SHAWL STOLE,Scarf_tradinginc,Women,2
1,4ACBEDBE977480D19B7B682D4878CAE2,Complete,2023-09-08 00:38:23.869168+00:00,2023-09-07 19:56:53.869168+00:00,2023-09-08 14:07:53.869168+00:00,NaT,2.5,F,4,1.1075,Accessories,Elegant PASHMINA SCARF WRAP SHAWL STOLE,Scarf_tradinginc,Women,2


In [49]:
merged_df.status.unique()

array(['Shipped', 'Complete', 'Processing', 'Returned', 'Cancelled'],
      dtype=object)

In [7]:
#trainsform dates to datetime
merged_df['created_at'] = pd.to_datetime(merged_df['created_at'])
merged_df['shipped_at'] = pd.to_datetime(merged_df['shipped_at'])
merged_df['delivered_at'] = pd.to_datetime(merged_df['delivered_at'])
merged_df['returned_at'] = pd.to_datetime(merged_df['returned_at'])

In [8]:
merged_df.head()
merged_df.groupby('sku').size().reset_index(name='sales')
# create column sales that is the number sales per sku
# Correct way to calculate sales per SKU
merged_df['sales'] = merged_df.groupby('sku')['num_of_item'].transform('sum')


In [9]:
merged_df.head(3
               )

,sku,status,created_at,shipped_at,delivered_at,returned_at,sale_price,gender,num_of_item,cost,category,name,brand,department,sales
0,4ACBEDBE977480D19B7B682D4878CAE2,Shipped,2023-08-20 21:21:15+00:00,2023-08-21 07:58:00+00:00,NaT,NaT,2.5,F,4,1.1075,Accessories,Elegant PASHMINA SCARF WRAP SHAWL STOLE,Scarf_tradinginc,Women,8
1,4ACBEDBE977480D19B7B682D4878CAE2,Complete,2023-09-08 00:38:23.869168+00:00,2023-09-07 19:56:53.869168+00:00,2023-09-08 14:07:53.869168+00:00,NaT,2.5,F,4,1.1075,Accessories,Elegant PASHMINA SCARF WRAP SHAWL STOLE,Scarf_tradinginc,Women,8
2,E0F19F64F086E393CEB0CF4A8C561B51,Shipped,2023-08-24 21:44:30+00:00,2023-08-21 07:58:00+00:00,NaT,NaT,64.0,F,4,30.2080,Shorts,Lilly Pulitzer Women's Callahan Short,Lilly Pulitzer,Women,13


In [10]:
merged_df.query('sku == "4ACBEDBE977480D19B7B682D4878CAE2"')
# TODO see duplicated skus with their corresponding sale and decide what to do with them

,sku,status,created_at,shipped_at,delivered_at,returned_at,sale_price,gender,num_of_item,cost,category,name,brand,department,sales
0,4ACBEDBE977480D19B7B682D4878CAE2,Shipped,2023-08-20 21:21:15+00:00,2023-08-21 07:58:00+00:00,NaT,NaT,2.5,F,4,1.1075,Accessories,Elegant PASHMINA SCARF WRAP SHAWL STOLE,Scarf_tradinginc,Women,8
1,4ACBEDBE977480D19B7B682D4878CAE2,Complete,2023-09-08 00:38:23.869168+00:00,2023-09-07 19:56:53.869168+00:00,2023-09-08 14:07:53.869168+00:00,NaT,2.5,F,4,1.1075,Accessories,Elegant PASHMINA SCARF WRAP SHAWL STOLE,Scarf_tradinginc,Women,8


In [17]:
from src.data.eda import Eda
eda = Eda()
eda.missing_values_table(merged_df)

The selected dataframe has 15 columns and 5 columns with missing values.


,Missing Values,% of Total Values
returned_at,162234,89.8
delivered_at,117399,65.0
shipped_at,63212,35.0
brand,142,0.1
name,12,0.0


In [21]:
# inspecting for seasonality
from src.data.seasonality import SeasonalityInspector, QuickSeasonalityInspector


# Initialize
inspector = SeasonalityInspector()
quick_inspector = QuickSeasonalityInspector()

# Load your data into a DataFrame called 'data'
# data = pd.read_csv('your_data.csv')

# Plot time series
#inspector.plot_time_series(merged_df, 'created_at', 'sales')



In [27]:
import numpy as np
# Find dominant period through FFT
dominant_period = quick_inspector.fast_fourier_transform(merged_df, 'sales')
print(f"Dominant period: {dominant_period}")


# Initialize default N
N = 7

# Check if dominant_period is finite
if np.isfinite(dominant_period):
    # Run rolling stats only if dominant_period is finite
    is_mean_stable, is_var_stable = quick_inspector.rolling_stats(merged_df, 'sales', window=int(dominant_period))
    
    # Update N based on rolling stats
    if not (is_mean_stable and is_var_stable):
        N = int(dominant_period)

print(f"Optimal N: {N}")


Dominant period: inf
Optimal N: 7


In [38]:
from src.data.feature_engineering import FeatureEngineeringProcess
fep = FeatureEngineeringProcess()


In [31]:
merged_df_fe1 = fep.price_sales_correlation_features_updated(merged_df, 7, [(-5.0, 1.0), (-3.0, 1.0), (-2.0, 1.0), (1.0, 1.0), (-1.0, 0.5), (-1.0, 0.33)],
                                                             'sku', 'created_at', 'sale_price', 'sales')

In [32]:
from src.data.eda import Eda
eda = Eda()

In [33]:
eda.missing_values_table(merged_df_fe1)

The selected dataframe has 21 columns and 11 columns with missing values.


,Missing Values,% of Total Values
returned_at,162234,89.8
f_corr_-5.0_1.0,148221,82.1
f_corr_-3.0_1.0,148221,82.1
f_corr_-2.0_1.0,148221,82.1
f_corr_1.0_1.0,148221,82.1
f_corr_-1.0_0.5,148221,82.1
f_corr_-1.0_0.33,148221,82.1
delivered_at,117399,65.0
shipped_at,63212,35.0
brand,142,0.1


In [37]:
merged_df_fe2 =  fep.normalize_features(merged_df_fe1, [5, 10, 15], 28, 'sku', 'created_at', 'sale_price', 'sales')

In [41]:
merged_df_fe3, insuficient_data = fep.filter_stability_periods(merged_df_fe2, 7, 0.04, sku_column='sku', date_column='created_at', price_column='sale_price')